In [ ]:
floor_area_poly

In [ ]:
# corrected_orientation_and_opening.py
import math
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString
from Pynite import FEModel3D

# -----------------------
# Parameters (editable)
# -----------------------
room_x = 3.00        # long direction (m)
room_y = 1.87        # short direction (m)
opening_x = 1.30     # opening size in x (m)
opening_y = 0.70     # opening size in y (m)

joist_width = 0.06
joist_depth = 0.12
bearing = 0.04       # end bearing into walls (m)
spacing = 0.40       # center-to-center joist spacing (m)

dead_kNpm2 = 0.5
live_kNpm2 = 2.0

E_mod = 1.0e10
G_mod = 3.8e9

# Convert loads to N/m^2
dead = dead_kNpm2 * 1000.0
live = live_kNpm2 * 1000.0



# -----------------------
# Create joist centerlines running along Y (short/span direction)
# We place joist centerlines at X positions across the long direction
# -----------------------

room_poly = Polygon([(0, 0), (room_x, 0), (room_x, room_y), (0, room_y)])
# number of joists to span the long direction (room_x)
num_joists = int(round(room_x / spacing)) + 1
# keep them inside the wall faces (avoid placing centerline inside wall thickness)
x_positions = np.linspace(joist_width/2.0, room_x - joist_width/2.0, num_joists)

joist_centers = []
for i, x in enumerate(x_positions):
    y_start = bearing
    y_end = room_y - bearing
    centerline = LineString([(x, y_start), (x, y_end)])
    joist_centers.append((f'J{i+1}', centerline))

# -----------------------
# Tributary polygons: vertical strips between midlines in X
# -----------------------
tributary_polys = {}
for idx, (name, line) in enumerate(joist_centers):
    x = line.coords[0][0]
    # left boundary
    if idx == 0:
        left = 0.0
    else:
        left = (x + joist_centers[idx-1][1].coords[0][0]) / 2.0
    # right boundary
    if idx == len(joist_centers)-1:
        right = room_x
    else:
        right = (x + joist_centers[idx+1][1].coords[0][0]) / 2.0

    strip_poly = Polygon([(left, 0), (right, 0), (right, room_y), (left, room_y)])
    trib = strip_poly.intersection(room_poly)
    tributary_polys[name] = trib

# -----------------------
# Compute distributed loads per joist (N/m along member)
# -----------------------
joist_info = []
for name, line in joist_centers:
    length = line.length
    trib_area = tributary_polys[name].area
    dead_total = dead * trib_area
    live_total = live * trib_area
    w_dead = dead_total / length if length > 0 else 0.0
    w_live = live_total / length if length > 0 else 0.0
    w_total = w_dead + w_live
    joist_info.append({
        'name': name,
        'line': line,
        'length': length,
        'tributary_area': trib_area,
        'w_dead_Npm': w_dead,
        'w_live_Npm': w_live,
        'w_total_Npm': w_total
    })

# -----------------------
# Build PyNite model (correct API usage)
# -----------------------
model = FEModel3D()
# add material and section to model
A = joist_width * joist_depth
Iy = (joist_width * joist_depth**3) / 12.0
Iz = (joist_depth * joist_width**3) / 12.0
J = 2 * (joist_width * joist_depth**3) / 12.0

model.add_material('MAT1', E=E_mod, G=G_mod, nu=0.3, rho=0.0)
model.add_section('RECT1', A=A, Iy=Iy, Iz=Iz, J=J)

# add nodes/members and loads
for j in joist_info:
    name = j['name']
    x = j['line'].coords[0][0]
    y1 = j['line'].coords[0][1]
    y2 = j['line'].coords[-1][1]
    n1 = f'{name}_N1'
    n2 = f'{name}_N2'
    model.add_node(n1, x, y1, 0.0)
    model.add_node(n2, x, y2, 0.0)
    model.add_member(name, i_node=n1, j_node=n2, material_name='MAT1', section_name='RECT1')
    # simple vertical supports at both ends (fix global Z)
    model.def_support(n1, support_DZ=True)
    model.def_support(n2, support_DZ=True)
    # apply uniform distributed load in global -Z direction
    model.add_member_dist_load(name, 'FZ', -j['w_total_Npm'], -j['w_total_Npm'], x1=0.0, x2=j['length'], case='D')

# run analysis with a simple combo '1.0DL' (dead only here; you can add live similarly)
model.add_load_combo('1.0DL', {'D': 1.0})
model.analyze_linear(log=False, check_stability=True, check_statics=False)

# -----------------------
# Print short summary and plot
# -----------------------
for j in joist_info:
    n1 = model.nodes[f"{j['name']}_N1"]
    n2 = model.nodes[f"{j['name']}_N2"]
    rxn1 = n1.RxnFZ.get('1.0DL', 0.0)
    rxn2 = n2.RxnFZ.get('1.0DL', 0.0)
    print(f"{j['name']}: len={j['length']:.3f} m, trib_area={j['tributary_area']:.3f} m^2, w_total={j['w_total_Npm']:.1f} N/m")
    print(f"   Reactions FZ @ {n1.name} = {rxn1:.1f}, @ {n2.name} = {rxn2:.1f}")

# Visualize room, opening, tributaries, and joist centerlines
fig, ax = plt.subplots(figsize=(8,5))
# room outline
rx, ry = room_poly.exterior.xy
ax.plot(rx, ry, color='white', linewidth=1.2)
# opening (red)
ox, oy = opening_poly.exterior.xy
ax.plot(ox, oy, color='red', linewidth=1.6)
# tributaries
for name, poly in tributary_polys.items():
    if poly.is_empty:
        continue
    x0, y0 = poly.exterior.xy
    ax.fill(x0, y0, alpha=0.35)
    cx, cy = poly.representative_point().x, poly.representative_point().y
    ax.text(cx, cy, name, ha='center', va='center', color='white', fontsize=8)
# centerlines
for j in joist_info:
    xs, ys = j['line'].xy
    ax.plot(xs, ys, color='white', linestyle='--')

ax.set_xlim(-0.05, room_x+0.05)
ax.set_ylim(-0.05, room_y+0.05)
ax.set_aspect('equal', 'box')
ax.set_facecolor('k')
ax.set_title('Room, Opening (red) and Joist Tributary Areas', color='white')
plt.show()


In [ ]:
import numpy as np

# --- Define Floor Plan Geometry ---
# Overall dimensions
floor_length = 20.0  # ft
floor_width = 12.0  # ft

# Opening dimensions and location
opening_width = 6.0  # ft
opening_length = 4.0  # ft
opening_x_start = 8.0  # ft

# Framing parameters
joist_spacing = 16.0 / 12.0  # 16 inches on center, converted to feet

# --- Define Structural Members ---
# Using a list of dictionaries to represent each member
framing = []

# Add regular joists before the opening
num_joists_before = int(opening_x_start / joist_spacing)
for i in range(num_joists_before + 1):
    x_pos = i * joist_spacing
    framing.append({
        "id": f"J{i+1}",
        "type": "joist",
        "start": [x_pos, 0],
        "end": [x_pos, floor_width]
    })

# Add members around the opening
header_1_x = opening_x_start
header_2_x = opening_x_start + opening_length

trimmer_1_x = framing[-1]["start"][0] # Double trimmer
trimmer_2_x = header_2_x + joist_spacing # Single trimmer next to wall

framing.append({"id": "H1", "type": "header", "start": [header_1_x, 0], "end": [header_1_x, opening_width]})
framing.append({"id": "H2", "type": "header", "start": [header_2_x, 0], "end": [header_2_x, opening_width]})

# Double trimmer joist (represented as two joists)
framing.append({"id": "T1_A", "type": "trimmer", "start": [trimmer_1_x, 0], "end": [trimmer_1_x, floor_width]})
framing.append({"id": "T1_B", "type": "trimmer", "start": [trimmer_1_x - 1.5/12.0, 0], "end": [trimmer_1_x - 1.5/12.0, floor_width]}) # Assuming 1.5" joist width

# Tail joists within the opening
num_tail_joists = int(opening_length / joist_spacing) -1
for i in range(num_tail_joists):
  x_pos = header_1_x + (i + 1) * joist_spacing
  framing.append({
      "id": f"TJ{i+1}",
      "type": "tail_joist",
      "start": [x_pos, 0],
      "end": [x_pos, opening_width]
  })


# Add regular joists after the opening
x_start_after = trimmer_2_x
num_joists_after = int((floor_length - x_start_after) / joist_spacing)
for i in range(num_joists_after + 1):
    x_pos = x_start_after + i * joist_spacing
    framing.append({
        "id": f"J{num_joists_before + i + 2}",
        "type": "joist",
        "start": [x_pos, 0],
        "end": [x_pos, floor_width]
    })

# The right-most trimmer is a single joist next to the wall
framing.append({"id": "T2", "type": "trimmer", "start": [trimmer_2_x, 0], "end": [trimmer_2_x, floor_width]})
framing

In [ ]:

from Pynite import FEModel3D
from Pynite import render_model

frame = FEModel3D()

# 2. Define Material Properties for Douglas Fir
# Density (rho): 530 kg/m^3
# Poisson's Ratio (nu): 0.42 (average value for Douglas Fir)
E = 11e9  # # Modulus of Elasticity (E) N/m^2
nu = 0.42
rho = 530 # kg/m^3
# Shear Modulus (G) is calculated from E and nu for isotropic materials
G = E / (2 * (1 + nu))
frame.add_material('DouglasFir', E, G, nu, rho)

# 3. Define Beam Cross-Sections
# Single joist: 0.06m width x 0.12m depth
# Double trimmer joist: 0.12m width x 0.12m depth
frame.add_rectangle_section('SingleJoist', 0.12, 0.06, 'DouglasFir')
frame.add_rectangle_section('DoubleTrimmer', 0.12, 0.12, 'DouglasFir')

# 4. Add Nodes to the model
# Node coordinates are in (X, Y, Z) format in meters
# Left side joists
frame.add_node('N1', 0.03, 0, 0)
frame.add_node('N2', 0.03, 1.87, 0)
frame.add_node('N3', 0.5467, 0, 0)
frame.add_node('N4', 0.5467, 1.87, 0)
frame.add_node('N5', 1.0634, 0, 0)
frame.add_node('N6', 1.0634, 1.87, 0)

# Right side framing for the opening
frame.add_node('N7', 1.58, 0, 0)       # Double trimmer bottom
frame.add_node('N8', 1.58, 0.585, 0)  # Double trimmer at header 1
frame.add_node('N9', 1.58, 1.285, 0)  # Double trimmer at header 2
frame.add_node('N10', 1.58, 1.87, 0)    # Double trimmer top
frame.add_node('N11', 2.97, 0, 0)      # Right trimmer bottom
frame.add_node('N12', 2.97, 0.585, 0) # Right trimmer at header 1
frame.add_node('N13', 2.97, 1.285, 0) # Right trimmer at header 2
frame.add_node('N14', 2.97, 1.87, 0)   # Right trimmer top
frame.add_node('N15', 2.0433, 0.585, 0)# Tail joist 1 on header 1
frame.add_node('N16', 2.0433, 1.285, 0)# Tail joist 1 on header 2
frame.add_node('N17', 2.5066, 0.585, 0)# Tail joist 2 on header 1
frame.add_node('N18', 2.5066, 1.285, 0)# Tail joist 2 on header 2

# 5. Add Members (beams and joists)
# Left side joists
frame.add_member('M1', 'N1', 'N2', 'SingleJoist')
frame.add_member('M2', 'N3', 'N4', 'SingleJoist')
frame.add_member('M3', 'N5', 'N6', 'SingleJoist')

# Double trimmer segments
frame.add_member('M4', 'N7', 'N8', 'DoubleTrimmer')
frame.add_member('M5', 'N8', 'N9', 'DoubleTrimmer')
frame.add_member('M6', 'N9', 'N10', 'DoubleTrimmer')

# Right trimmer segments
frame.add_member('M7', 'N11', 'N12', 'SingleJoist')
frame.add_member('M8', 'N12', 'N13', 'SingleJoist')
frame.add_member('M9', 'N13', 'N14', 'SingleJoist')

# Header 1 segments (framing the opening)
frame.add_member('M10', 'N8', 'N15', 'SingleJoist')
frame.add_member('M11', 'N15', 'N17', 'SingleJoist')
frame.add_member('M12', 'N17', 'N12', 'SingleJoist')

# Header 2 segments (framing the opening)
frame.add_member('M13', 'N9', 'N16', 'SingleJoist')
frame.add_member('M14', 'N16', 'N18', 'SingleJoist')
frame.add_member('M15', 'N18', 'N13', 'SingleJoist')

# Tail joists within the opening
frame.add_member('M16', 'N15', 'N16', 'SingleJoist')
frame.add_member('M17', 'N17', 'N18', 'SingleJoist')

# 6. Define Supports
# Add pinned supports at the ends of the vertical joists, as they rest in wall indentations
supported_nodes = ['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N10', 'N11', 'N14']
for node in supported_nodes:
    frame.def_support(node, True, True, True, False, False, False)

# 7. Add Loads
# Add the self-weight of all members as a load case to see its effect
frame.add_self_weight('SW')

# 8. Analyze the model
frame.analyze(check_stability=True)

# 9. Visualize the results
try:
    # Render the 3D model with the deformed shape
    render_model(frame, text_height=0.1, deformed_shape=True, deformed_scale=100, render_loads=True)
    print("Successfully rendered the model. Please check the plot window.")
    print("The plot shows the deformed shape under self-weight (scaled by 100).")
except Exception as e:
    print(f"Could not render the model. Error: {e}")
    print("Rendering requires VTK and PyVista. Proceeding without visualization.")
finally:
    # Print the maximum deflection in the Z direction
    max_deflection = min(frame.Nodes[node].DZ['SW'] for node in frame.Nodes if 'SW' in frame.Nodes[node].DZ)
    print(f"\nMaximum deflection in Z-direction due to self-weight: {max_deflection*1000:.4f} mm")

In [ ]:
from Pynite import FEModel3D
from Pynite.Rendering import Renderer

# Create a new model
model = FEModel3D()

# Define material properties for wood
E = 11e9  # N/m^2
G = 690e6  # N/m^2
nu = 0.3
rho = 500  # kg/m^3
model.add_material('Wood', E, G, nu, rho)

# Define section properties for the beams
# Single joist: 0.06m x 0.12m
A = 0.06 * 0.12
Iz = 0.06 * 0.12**3 / 12
Iy = 0.12 * 0.06**3 / 12
J = Iy + Iz
model.add_rectangle_mesh('SingleJoist', 0.12, 0.06, 'Wood')

# Double joist: 0.12m x 0.12m
A = 0.12 * 0.12
Iz = 0.12 * 0.12**3 / 12
Iy = 0.12 * 0.12**3 / 12
J = Iy + Iz
model.add_rectangle_mesh('DoubleJoist', 0.12, 0.12, 'Wood')

print("Model, material, and sections created successfully.")

In [ ]:
# Corrected script for modeling the floor framing plan with PyNiteFEA

# Note: You must have PyNiteFEA installed in your environment.
# You can install it using: pip install PyNiteFEA

from Pynite import FEModel3D
from Pynite.Rendering import Renderer

# 1. Create a new model
frame_model = FEModel3D()

# 2. Define material properties for wood
# Using typical properties for pine or fir
E = 11e9  # Modulus of Elasticity (Pa)
G = 690e6 # Shear Modulus (Pa)
nu = 0.3  # Poisson's Ratio
rho = 500 # Density (kg/m^3)
frame_model.add_material('Wood', E, G, nu, rho)

# --- CORRECTED SECTION DEFINITIONS ---
# Define and add the single joist section
D_single = 0.12 # depth (m)
B_single = 0.06 # width (m)
A_single = D_single * B_single # Cross-sectional area
Iz_single = (B_single * D_single**3) / 12 # Moment of inertia about z-axis
Iy_single = (D_single * B_single**3) / 12 # Moment of inertia about y-axis
J_single = Iy_single + Iz_single # Torsion constant (approximation for rectangle)
frame_model.add_section('SingleJoist', A_single, Iy_single, Iz_single, J_single)

# Define and add the double joist section
D_double = 0.12 # depth (m)
B_double = 0.12 # width (m)
A_double = D_double * B_double # Cross-sectional area
Iz_double = (B_double * D_double**3) / 12 # Moment of inertia about z-axis
Iy_double = (D_double * B_double**3) / 12 # Moment of inertia about y-axis
J_double = Iy_double + Iz_double # Torsion constant
frame_model.add_section('DoubleJoist', A_double, Iy_double, Iz_double, J_double)


# Add nodes to the model
# Room dimensions: 3m (X) by 1.87m (Y)
# Opening dimensions: 1.3m (X) by 0.7m (Y)

# Far left joist (at x=0)
frame_model.add_node('N1', 0, 0, 0)
frame_model.add_node('N2', 0, 1.87, 0)

# Two intermediate joists on the left
# Spacing is (3.0 - 1.3) / 3 = 0.567m
frame_model.add_node('N3', 0.567, 0, 0)
frame_model.add_node('N4', 0.567, 1.87, 0)
frame_model.add_node('N5', 1.133, 0, 0)
frame_model.add_node('N6', 1.133, 1.87, 0)

# Double trimmer joist (left side of opening)
frame_model.add_node('N7', 1.7, 0, 0)
frame_model.add_node('N8', 1.7, 1.87, 0)

# Single trimmer joist (right side of opening, hugs the wall)
frame_model.add_node('N9', 3.0, 0, 0)
frame_model.add_node('N10', 3.0, 1.87, 0)

# Nodes for headers and tail joists
# Opening is from Y = (1.87-0.7)/2 = 0.585 to Y = 0.585 + 0.7 = 1.285
frame_model.add_node('N11', 1.7, 0.585, 0) # Connection point on double trimmer
frame_model.add_node('N12', 3.0, 0.585, 0) # Connection point on right trimmer
frame_model.add_node('N13', 1.7, 1.285, 0) # Connection point on double trimmer
frame_model.add_node('N14', 3.0, 1.285, 0) # Connection point on right trimmer

# Add members to the model, referencing the sections by name
# Joists on the left side
frame_model.add_member('M1', 'N1', 'N2', 'Wood', 'SingleJoist')
frame_model.add_member('M2', 'N3', 'N4', 'Wood', 'SingleJoist')
frame_model.add_member('M3', 'N5', 'N6', 'Wood', 'SingleJoist')

# Trimmer joists for the opening
frame_model.add_member('M4', 'N7', 'N8', 'Wood', 'DoubleJoist') # Double trimmer
frame_model.add_member('M5', 'N9', 'N10', 'Wood', 'SingleJoist') # Single trimmer

# Header joists
frame_model.add_member('H1', 'N11', 'N12', 'Wood', 'SingleJoist')
frame_model.add_member('H2', 'N13', 'N14', 'Wood', 'SingleJoist')

# Tail joists (short joists supporting the opening)
frame_model.add_member('T1', 'N1', 'N11', 'Wood', 'SingleJoist') # This seems incorrect based on description, connecting to N11 and N13.
# The description says "two tail joists, one for each header". I will model them going from the header to the trimmer.
# The prompt is slightly ambiguous here. I will model short members from the headers to the adjacent vertical members.
# Given the setup, the tail joists would logically run parallel to the main joists.
# I will add tail joists from the wall to the headers.
frame_model.add_node('N15', 1.7, 0, 0) # Reusing N7
frame_model.add_node('N16', 3.0, 0, 0) # Reusing N9
frame_model.add_member('Tail1', 'N15', 'N11', 'Wood', 'SingleJoist')
frame_model.add_member('Tail2', 'N16', 'N12', 'Wood', 'SingleJoist')


# Define supports (assuming fixed connections at top and bottom walls)
# Note: Header joists are supported by trimmers, not the wall, so they don't get supports.
for i in range(1, 11):
    frame_model.def_support(f'N{i}', True, True, True, True, True, True)


# # Render the model to visualize it
# renderer = Renderer(frame_model)
# renderer.render_model()

In [ ]:
# Import the modeling class
from Pynite import FEModel3D

# Create a new model
model = FEModel3D()

# Geometry
model.add_node('N1', 0.0, 0.0, 0.0)
model.add_node('N2', 3.0, 0.0, 0.0)

E = 11e9  # Modulus of Elasticity (Pa)
G = 690e6 # Shear Modulus (Pa)
nu = 0.3  # Poisson's Ratio
rho = 500 # Density (kg/m^3)
model.add_material('wood', E=E, G=G, nu=nu, rho=rho)
model.add_section('Wsect', A=10.0, Iy=100.0, Iz=200.0, J=5.0)

m1 = model.add_member('M1', i_node='N1', j_node='N2', material_name='wood', section_name='Wsect')

# Supports and loads
model.def_support('N1', support_DX=True, support_DY=True, support_DZ=True, support_RY=True, support_RZ=True)
model.add_node_load('N2', direction='FZ', P=-5.0, case='D')

# Load combinations and analysis
model.add_load_combo('1.0D', {'D': 1.0})
model.analyze_linear(log=False)

# Results
uz = model.nodes['N2'].DZ['1.0D']
rxn = model.nodes['N1'].RxnFZ['1.0D']

In [ ]:
from Pynite.Rendering import Renderer

# Create a `Renderer` for this model
my_rndr = Renderer(model)

# Set the load combination to be rendered
my_rndr.combo_name = '1.0D'

# Now, render the model
my_rndr.render_model()